In [1]:
import os
import json
import sqlalchemy
from sqlalchemy import create_engine
import pandas as pd
from io import StringIO

In [2]:
# load environment
postgres_config = None
with open(os.path.join(os.path.expanduser('~'), '.postgres_connection.json'), 'r') as postgres_config:
    postgres_config = json.load(postgres_config)


In [8]:
# create db connection
dbname = 'BeerIsLife'
user = postgres_config['user']
password = postgres_config['password']
host = postgres_config['host']

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{dbname}?client_encoding=utf8")

### helper functions

In [9]:
def get_file_contents(file_name):
    """
    Returns Pandas Dataframe
    """
    df = None
    with open(os.path.join('.', 'data', file_name), 'r', encoding='utf8') as fd:
        df = pd.read_csv(fd)
    return df

### Beer Reviews

In [10]:
# load beer reviews
beer_reviews_df = get_file_contents('beer_reviews.csv')

In [11]:
# clean the data
beer_reviews_df.beer_abv = beer_reviews_df.beer_abv.replace("", None)
beer_reviews_df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [ ]:
# load data to table
beer_reviews_df.to_sql('beerreviews', engine, if_exists='replace', index=False, method = 'multi')

## happiness and alcohol consumption

In [34]:
# load alcohol consumption
alcohol_consumption_df = get_file_contents('HappinessAlcoholConsumption.csv')
alcohol_consumption_df.head()

,Country,Region,Hemisphere,HappinessScore,HDI,GDP_PerCapita,Beer_PerCapita,Spirit_PerCapita,Wine_PerCapita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97


In [39]:
#clean up columns
alcohol_consumption_df = alcohol_consumption_df.rename(columns={
    'Country': 'country',
    'Region': 'region',
    'Hemisphere': 'hemisphere',
    'HappinessScore': 'happiness_score',
    'HDI': 'hdi',
    'GDP_PerCapita': 'gdp_per_capita',
    'Beer_PerCapita': 'beer_per_capita',
    'Spirit_PerCapita': 'spirit_per_capita',
    'Wine_PerCapita': 'wine_per_capita'
}).copy()
alcohol_consumption_df.head()

,country,region,hemisphere,happiness_score,hdi,gdp_per_capita,beer_per_capita,spirit_per_capita,wine_per_capita
0,Denmark,Western Europe,north,7.526,928,53.579,224,81,278
1,Switzerland,Western Europe,north,7.509,943,79.866,185,100,280
2,Iceland,Western Europe,north,7.501,933,60.530,233,61,78
3,Norway,Western Europe,north,7.498,951,70.890,169,71,129
4,Finland,Western Europe,north,7.413,918,43.433,263,133,97


In [41]:
# load data to sql
alcohol_consumption_df.to_sql('alcohol_consumption', engine, if_exists='replace', index=False, method = 'multi')